## Discrete states of DNA

Nucleic acids (deoxyribonucleic acid [DNA] and its ribonucleic version [RNA])  are composed of units called nucleotides. In DNA, each nucleotide is conformed by one deoxyribose sugar, one phosphate group, and one of four nitrogen nucleobases (adenine [A], cytosine [C], guanine [G], or thymine [T]). In the case of RNA, the sugar is ribose and the thymine base is replaced by uracil (U). These nucleotides are classified into two main chemical groups, purines and pyrimidines. In DNA, two chains are bound together by a series of hydrogen bonds between a specific purine and its complementary pyrimidine. Adenine pairs with thymine and guanine with cytosine. 

Variability in this configuration is broad but rare, and we can find some non-canonical bases, bounds, or even rearrangement in the whole structure. Regardless, as we see, DNA offers a wide variety of discrete characters to incorporate in models to study it or predict patterns.
For inferring phylogenies, the nitrogen nucleobase (A, C, G, T) has been one of the primary sources of information when comparing sequences and calculating distances; nevertheless, we have some hypothetical models that incorporate other information to predict the distribution in a given sequence.

DNA is constely exposed to different phenomena that may change or mutate their sequence, with puntual mutations (mutations that change one base for another) or much larger mutations. Mutation can be caused by multiple situations (UV exposition, errors during replication or celular division, insertion or delation or segments), and  even being rare, they play an importal role in organisms evolution. They are accumulable in time and from it we can infer ancestry-decendent relationships. 

Considering puntual mutations, we can illustrate these possible discrete states (nucleotides) and their possible changes using the following simple graph.

In [4]:
import toyplot
import itertools
import random

edges = [i for i in itertools.permutations('ACGT', r=2)]
transitions = {("A","G"),("C","T")}
colors = []
for e in edges:
    sorted_e = tuple(sorted(list(e)))
    if sorted_e in transitions:
        colors.append("blue")
    else:
        colors.append("red")

        
canvas, axes, graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        ecolor=colors,
        ewidth=5,
        eopacity=0.35,
        width=350,
        height=350,
        margin=0,
        tmarker=">", 
        vsize=50,
        vcoordinates=[(-1,1),(-1,-1),(1,1), (1,-1)],
        vstyle={"stroke": "black", "stroke-width": 2, "fill": "none"},
        vlstyle={"font-size": "20px"},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges())
)

f_style = {"font-size": "14px"} 
canvas.text(175,40, "Transitions", style=f_style)
canvas.text(175,310, "Transitions", style=f_style)
canvas.text(40,175, "Transversions", angle=90, style=f_style)
canvas.text(175,175, "Transversions", style=f_style)
canvas.text(310,175, "Transversions", angle=270, style=f_style);
canvas.style['background-color'] = 'white'

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="350.0px" viewBox="0 0 350.0 350.0" preserveAspectRatio="xMidYMid meet" style="background-color:white;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfa70806ab4504d72a8cee3c741875500"> A C G T Transitions Transitions Transversions Transversions Transversions

These discrete states and possible changes can be modeled using Markov chains to predict some expected patterns in the sequences.

## Markov chains

A Markov chain is a a system that describe the transition (do not confuse with nucleotide transitions) from one state to another considering only some simple probabilistic rules. In this stochastic model every change is only conditioned of the previous state, this is known as the *Markov property*. In terms of nucleotides, the probability of having a given nucleotide depends only of the previous nucleotide.

In this way we can consider that Markov chains are "memoryless" process, in which only the previous state can affect the new state, while the previous states are forgotten. This principle is true and only true in 1st-order Markov chains. As expected, there are high-level order Markov chains that extend the "memory" to a given number of previous states; however, in terms of substitutions models, 1st-order is the type of chain that is commonly used.

In the following animated graph, we can see how is a random walk in a Markov chain of 4 states.

<div class="markov">
  <iframe width="800" height="500" src="https://setosa.io/markov/index.html#%7B%22tm%22%3A%5B%5B0.5%2C0.2%2C0.1%2C0.2%5D%2C%5B0.1%2C0.5%2C0.3%2C0.1%5D%2C%5B0.05%2C0.2%2C0.7%2C0.05%5D%2C%5B0%2C0.05%2C0.05%2C0.9%5D%5D%7D"></iframe>
</div>

One crucial part of these models is the transitions probabilities between states. These can be stored and represented as a matrix (symmetrical or asymmetrical). In this matrix every row would sum 1.

We can declare this matrix in Python using a Numpy array as follows:

In [1345]:
import numpy as np
#                                A     C     G     T
transition_matrix = np.array([[0.50, 0.20, 0.10, 0.20],  # A
                              [0.10, 0.50, 0.30, 0.10],  # C
                              [0.05, 0.20, 0.70, 0.05],  # G
                              [0.00, 0.05, 0.05, 0.90]   # T
                             ])

Using the transition probabilities in this matrix we can do a random walk changing from one state to another a given number of steps. In the next function we implemented a simple way to do this and store each state visited in this Markov chain.

In [6]:
def print_random_walk(steps: int, 
                      initial_state: str, 
                      transition_matrix: np.ndarray,
                      states: str = "ACGT") -> str:
    """Do a random walk and return the sequence of all states visited
    
    Parameters
    ----------
    states
        States to consider in the Markov chain
    steps
        Number of steps in the random walk
    initial_state
        Where to start the random walk
    transition_matrix: np.array
        Transition matrix with transition probabilities for all states

    Returns
    -------
    Sequence of all states visited
        
    """
    current_state = initial_state
    result_sequence = [current_state]

    step = 1
    while step < steps:
        previous_state = current_state
        previous_idx = states.index(previous_state)
        
        # Choice one random base on the probabilities on the matrix picking the previous row 
        current_state = np.random.choice(list(states), p=transition_matrix[previous_idx])
        result_sequence.append(current_state)
        step += 1
    
    return "".join(result_sequence)

With this random walk function, we can generate a random sequence that follows the transition probabilities we stated in our matrix transition:

In [7]:
sequence = print_random_walk(states="ACGT", 
                  steps=200, 
                  initial_state="G", 
                  transition_matrix=transition_matrix)

sequence

'GGGGCGCCCTTTCGGGGAATTTTTTTTTTTTCCGGCAAAGGGGCCTTTTTTCTTTTTTTTTTTCCTTTTTTTTTTCCGGCCAAATTTTTTCCCCCGTTTTTCAATTTTTTTTTTTTTTTTTTTTCCCGGGCCAACCGGGAAACCCATTTTCGGGGCCCCCCCCCGTTTTTGGCCCCCGGGGGGGGCGTTTTTTTTTTTTT'

We can notice that this random sequence shows some patterns, for example more T's than A's, a similar number of C's and G's, and islands of certain bases. The transition probabilities we set influence these patterns notoriusly, and at some point the distribution of the states (in this case nucleotides) are predictable. If the sequence of changes is long enough we can predict how many T's, for example, we can have in a random sequence that follows a particular model. 

That predictability is represented by the stationary distribution or equilibrium in a Markov model. 

### Stationary distributions

Stationary distributions, also known as the equilibrium, are probability distributions that remain unchanged as time progresses. In other words, the distribution of every state in a sequence of changes tends to converge with time, and it will be the same after some point.

These stationary distributions are vectors with the distribution of all states in the model and are commonly denoted with π. They constitute an essential part of implementing models for calculating expected distributions in DNA sequences.

We can see this when we do several random walks and start noticing that the number of T's, for example, is very similar from one sequence to another. 
Let us generate 10 sequences with 1000 bases by doing different random walks and counting the number of T's we have in each of them.

In [8]:
for i in range(10):
    sequence = print_random_walk(states="ACGT", 
                      steps=1000, 
                      initial_state="G", 
                      transition_matrix=transition_matrix)
    n_ts = sequence.count("T")
    print(f"Seq. {i}: {sequence[0:10]}...{sequence[-10:-1]} - T's = {n_ts}")

Seq. 0: GGCGGGGGGG...GTTTTTTTT - T's = 430
Seq. 1: GTTTGGGGGG...TTTTTTTTT - T's = 461
Seq. 2: GGGCCGGGGG...GGCGGAAAA - T's = 423
Seq. 3: GAGGGGGGGG...TTTTTGGCC - T's = 389
Seq. 4: GGCCCCGTTT...CGCTTTTTT - T's = 535
Seq. 5: GCAAAAAACG...TTTTTTTTT - T's = 453
Seq. 6: GGGGGGGCCG...CCAAGGGAA - T's = 484
Seq. 7: GGGCCCCGGG...AAAACCCCC - T's = 437
Seq. 8: GGGGCCAAAA...TTTTTGGGG - T's = 411
Seq. 9: GGTTTTTTTT...CGCCCGGGG - T's = 549


We can see that most of them have round 450 T's (about 45%) of the entire sequence. If the sequence is long enough, we will end with the stationary distribution, and the number of T's would be predicted.

Let me illustrate this convergency with a plot where we ran for 10000 steps a random walk and tracked the frequency of each nucleotide.

In [9]:
import toyplot

def plot_convergency(states, steps, initial_state, transition_matrix):

    current_state = initial_state
    sequence = [current_state]
    empiric_frequencies = np.zeros((int(steps), len(states))) 

    step = 1
    while step < steps:
        previous_state = current_state
        previous_idx = states.index(previous_state)

        current_state = np.random.choice(list(states), p=transition_matrix[previous_idx])
        sequence.append(current_state)

        step_freq = []
        for base in states:
            step_freq.append(sequence.count(base) / len(sequence))

        empiric_frequencies[step] = step_freq
        step += 1


    label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px"}
    canvas, axes, mark = toyplot.plot(empiric_frequencies, ylabel="Frequency", xlabel="Steps")
    canvas.style['background-color'] = 'white'
    
    for i in states:
        axes.text(steps, empiric_frequencies[-1,states.index(i)], i, style=label_style)
    
    return canvas, axes, mark

In [10]:
plot_convergency(states="ACGT", initial_state="A", steps= 10000, transition_matrix=transition_matrix);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:white;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t818e2a456414467b924aa96bed70c588"> <path d="M 50.0 550.0 L 50.04860635848939 216.66666666666669 L 50.09721271697879 327.77777777777777 L 50.145819075468175 383.33333333333337 L 50.19442543395756 416.6666666666667 L 50.243031792446956 438.88888888888886 L 50.29163815093635 454.76190476190476 L 50.340244509425744 466.6666666666667 L 50.38885086791514 475.9259259259259 L 50.43745722640453 483.33333333333337 L 50.48606358489392 489.3939393939394 L 50.534669943383314 494.4444444444444 L 50.58327630187271 498.71794871794873 L 50.6318826603621 502.3809523809524 L 50.68048901885149 505.55555555555554 L 50.729095377340876 508.3333333333333 L 50.77770173583028 510.7843137254902 L 50.826308094319664 512.9629629629629 L 50.87491445280906 514.9122807017544 L 50.92352081129845 516.6666666666667 L 50.972127169787846 518.2539682539682 L 51.02073352827723 489.3939393939394 L 51.069339886766635 492.02898550724643 L 51.11794624525602 494.4444444444444 L 51.16655260374541 496.66666666666663 L 51.2151589622348 498.71794871794873 L 51.2637653207242 500.6172839506173 L 51.312371679213584 502.3809523809524 L 51.36097803770298 504.02298850574715 L 51.40958439619237 505.55555555555554 L 51.45819075468176 506.98924731182797 L 51.50679711317115 508.3333333333333 L 51.555403471660554 509.59595959595964 L 51.60400983014994 510.7843137254902 L 51.652616188639335 511.9047619047619 L 51.70122254712872 512.9629629629629 L 51.749828905618116 513.9639639639639 L 51.79843526410751 514.9122807017544 L 51.8470416225969 515.8119658119658 L 51.8956479810863 516.6666666666667 L 51.944254339575686 517.479674796748 L 51.99286069806507 518.2539682539682 L 52.04146705655447 518.9922480620155 L 52.09007341504386 519.6969696969697 L 52.138679773533255 520.3703703703703 L 52.18728613202265 521.0144927536231 L 52.23589249051204 521.6312056737588 L 52.28449884900143 522.2222222222223 L 52.333105207490824 522.7891156462584 L 52.38171156598022 523.3333333333333 L 52.430317924469605 523.8562091503268 L 52.478924282959 524.3589743589744 L 52.52753064144839 524.8427672955975 L 52.57613699993778 525.3086419753087 L 52.624743358427175 525.7575757575758 L 52.67334971691656 526.1904761904761 L 52.72195607540596 526.6081871345028 L 52.77056243389536 527.0114942528736 L 52.819168792384744 527.4011299435028 L 52.86777515087414 527.7777777777777 L 52.91638150936353 528.1420765027322 L 52.96498786785292 528.494623655914 L 53.01359422634232 528.8359788359788 L 53.06220058483171 529.1666666666667 L 53.110806943321094 529.4871794871796 L 53.15941330181049 529.7979797979797 L 53.20801966029988 530.0995024875622 L 53.25662601878927 530.3921568627452 L 53.305232377278664 530.6763285024155 L 53.353838735768065 530.952380952381 L 53.40244509425745 531.2206572769952 L 53.451051452746846 531.4814814814815 L 53.49965781123623 531.7351598173516 L 53.54826416972563 531.981981981982 L 53.59687052821502 532.2222222222223 L 53.64547688670441 532.4561403508773 L 53.6940832451938 532.6839826839827 L 53.742689603683196 532.9059829059829 L 53.79129596217258 533.1223628691982 L 53.83990232066198 533.3333333333333 L 53.88850867915137 525.3086419753087 L 53.937115037640766 525.609756097561 L 53.98572139613015 525.9036144578313 L 54.034327754619554 526.1904761904761 L 54.08293411310894 526.4705882352941 L 54.131540471598335 526.7441860465117 L 54.18014683008773 527.0114942528736 L 54.228753188577116 519.6969696969697 L 54.27735954706651 512.5468164794007 L 54.3259659055559 505.55555555555554 L 54.37457226404529 498.7179487179

We can find these stationary distributions using different approaches. One of them is using a Monte Carlo approach in which we do a random walk long enough to get the distribution from the actual sequence of events. This method is easy to understand but it is not very efficient.

In [11]:
def get_stationary_dist_monte_carlo(states, steps, initial_state, transition_matrix):
    # Create empty array to put the frequency of a given state
    frequencies = np.zeros(len(states)) 
    
    # do first step
    initial_idx = states.index(initial_state)
    frequencies[initial_idx] = 1
    previous_idx = initial_idx
    
    # do other steps
    step = 1
    while step < steps:
        current_state = np.random.choice(list(states), p=transition_matrix[previous_idx])
        current_idx = states.index(current_state)
        frequencies[current_idx] += 1
        previous_idx = current_idx
        step += 1
    
    stationary_distribution = frequencies / steps   # π
    return stationary_distribution


Using this method we can get the stationary distribution of our Markov chain:

In [12]:
get_stationary_dist_monte_carlo(states="ACGT", 
                          steps=1e6, 
                          initial_state="A", 
                          transition_matrix=transition_matrix)

array([0.066472, 0.186481, 0.286193, 0.460854])

With this, we can see that the frequency of T's will be 0.46 in any sequence produced by a random walk in this Markov chain. 

As we mentioned before, there are other methods to find this distribution; some are more efficient than the previous one. The following apply a repeated multiplication matrix method over the substitution matrix. Notice that it is still an iterative operation, but the iterations needed to reach the stationary distribution are fewer.

In [13]:
def get_stationary_dist_matrix_multiplication(states, iterations, transition_matrix):
    """Repeated matrix multiplication method"""
    current_matrix = transition_matrix
    
    iteration = 1
    while iteration < iterations:
        # Calculate the Matrix product of two arrays
        current_matrix = np.matmul(current_matrix, transition_matrix)
        iteration += 1
    
    stationary_distribution = current_matrix[0]
    return stationary_distribution

In [14]:
get_stationary_dist_matrix_multiplication("ACGT", iterations=100, transition_matrix=transition_matrix)

array([0.06593407, 0.18681319, 0.28571429, 0.46153846])

### Rate matrices

Here we can introduce another concept, a rate matrix (commonly called Q matrix) in which each element describes the rate (the expected number of events per some time) of one state change into other states. Different of transition matrix, in a rate matrix every row will sum 0.


In [1368]:
# First we can declare our rate matrix (Q)
#                         A     C     G     T
rate_matrix = np.array([[-1.00, 0.333, 0.333, 0.333],  # A
                        [0.333, -1.00, 0.333, 0.333],  # C
                        [0.333, 0.333, -1.00, 0.333],  # G
                        [0.333, 0.333, 0.333, -1.00]   # T
                       ])

The library `scipy` includes a library with lineal algebra operation in which we can use matrix exponentiation en order to get the probabilities of change at a given a time. Take into account that if `time`is large enough you will end getting the stationary distribution (this is another way to get this vector other than Monte Carlo or repeated matrix multiplication approaches). 

Because this method works well with real numbers (previous approaches only worked well with integers [steps or iterations]), we can calculate the probability of transition given a branch length.

In [1383]:
from scipy.linalg import expm

time = 0.1
expm(Q * time)

array([[0.47589566, 0.17470145, 0.17470145, 0.17470145],
       [0.17470145, 0.47589566, 0.17470145, 0.17470145],
       [0.17470145, 0.17470145, 0.47589566, 0.17470145],
       [0.17470145, 0.17470145, 0.17470145, 0.47589566]])

## Simulation of evolution under a markov model

In [ ]:
#test

In [483]:
import numpy as np
#                                A     C     G     T
transition_matrix = np.array([[0.01, 0.33, 0.33, 0.33],  # A
                              [0.33, 0.01, 0.33, 0.33],  # C
                              [0.33, 0.33, 0.01, 0.33],  # G
                              [0.33, 0.33, 0.33, 0.01]   # T
                             ])

In [1335]:
import numpy as np
rate = 0.33
#                                A     C     G     T
transition_matrix = np.array([[1-3*rate, rate, rate, rate],  # A
                              [rate, 1-3*rate, rate, rate],  # C
                              [rate, rate, 1-3*rate, rate],  # G
                              [rate, rate, rate, 1-3*rate]   # T
                             ])
transition_matrix

array([[0.01, 0.33, 0.33, 0.33],
       [0.33, 0.01, 0.33, 0.33],
       [0.33, 0.33, 0.01, 0.33],
       [0.33, 0.33, 0.33, 0.01]])

In [1336]:
import copy

class lineage:
    STATES = [0,1,2,3]
    BASES = "ACGT"
    
    def __init__(self, transition_matrix, length=1000, tips=4, parent=None, states=STATES):
        if not parent:
            self.sequence = np.random.choice(states, size=length)
        else:
            self.sequence = parent.sequence.copy()
            
#         self.ancestral_sequence = self.sequence
        self.length = length
        self.transition_matrix = transition_matrix
        self.parent = parent
        self.generations = 0
        self.right = None
        self.left = None
        self.real_changes = np.zeros(length)
          
    def evolve(self, generations=1):
        for gen in range(int(generations)):          
            for index, base in enumerate(self.sequence):
                new_base = np.random.choice(STATES, p=self.transition_matrix[base])
                if new_base != base:
                    self.sequence[index] = new_base
                    self.real_changes[index] += 1
        
            self.generations += 1
       
    def cladogenesis(self):
        self.right = lineage(self.transition_matrix, length=self.length, parent=self)
        self.left = lineage(self.transition_matrix, length=self.length, parent=self)
        return self.left, self.right
    
    def translate_sequence(self):
        return [BASES[b] for b in self.sequence]


In [1337]:
ol = lineage(transition_matrix, length=100)
# ol.sequence

In [1338]:
ol.evolve(500)
# ol.sequence

ch1, ch2 = ol.cladogenesis()
# ch1.sequence, ch2.sequence

ch1.evolve(2000)
ch2.evolve(2000)
# ch1.sequence, ch2.sequence

In [1339]:
# ch1.translate_sequence(), ch2.translate_sequence()

In [1340]:
# ch1.real_changes, ch2.real_changes

In [1341]:
p_distance = np.equal(ch1.sequence, ch2.sequence).sum() / ch1.length
p_distance

0.26

In [1342]:
corrected_distance_jc = -3/4 * np.log(1 - 4/3 * p_distance)
corrected_distance_jc

0.31925086156926286

In [ ]:
### Tracking distances along time

In [1343]:
generations = 10000
results = np.zeros((generations, 2))
ol = lineage(transition_matrix, length=1000)
ol.evolve(100)
ch1, ch2 = ol.cladogenesis()

for g in range(generations):
    ch1.evolve()
    ch2.evolve()
    p_distance = np.equal(ch1.sequence, ch2.sequence).sum() / ch1.length
    
    # if distances is greater than 3/4 the equation does not exist
    if p_distance < 3/4:
        # correct distance using jc model
        corrected_distance_jc = -3/4 * np.log(1 - 4/3 * p_distance)
    else:
        corrected_distance_jc = np.inf

    results[g] = [corrected_distance_jc, p_distance]
    
    
canvas, axes, mark = toyplot.plot(results, ylabel="Distance", xlabel="Generations");
axes.text(generations, results[-1][0], "With model", style={"text-anchor":"start", "-toyplot-anchor-shift":"5px"});
axes.text(generations, results[-1][1], "Without model", style={"text-anchor":"start", "-toyplot-anchor-shift":"5px"});

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf89566985be0466c89308e74da94b2be"> <path d="M 50.0 50.0 L 50.04166666666667 293.66367379044027 L 50.083333333333336 384.0774291038743 L 50.12500000000001 324.33930199473684 L 50.166666666666664 279.8502415299612 L 50.208333333333336 400.30527748525276 L 50.25 255.4031606270123 L 50.291666666666664 290.2208300403511 L 50.33333333333333 393.8327256220751 L 50.375 320.95812115885536 L 50.41666666666667 351.1492122573612 L 50.458333333333336 384.0774291038743 L 50.5 286.7709957397782 L 50.541666666666664 347.8210176391559 L 50.583333333333336 307.36570619123654 L 50.625 258.91721112332203 L 50.66666666666667 265.9235831521652 L 50.708333333333336 230.5991754460356 L 50.75000000000001 327.7137674720065 L 50.791666666666664 380.8131767851065 L 50.83333333333333 310.77401788444627 L 50.875 244.817189875185 L 50.916666666666664 290.2208300403511 L 50.958333333333336 237.72305510403544 L 51.0 331.0815442125284 L 51.04166666666667 258.91721112332203 L 51.083333333333336 279.8502415299612 L 51.125 303.95054361564286 L 51.166666666666664 290.2208300403511 L 51.208333333333336 334.4426586801737 L 51.25 258.91721112332203 L 51.291666666666664 334.4426586801737 L 51.333333333333336 337.7971371820668 L 51.375 198.1668327167817 L 51.416666666666664 248.35318034875888 L 51.45833333333333 324.33930199473684 L 51.5 334.4426586801737 L 51.541666666666664 324.33930199473684 L 51.583333333333336 244.817189875185 L 51.625 347.8210176391559 L 51.66666666666667 227.0260031315936 L 51.708333333333336 307.36570619123654 L 51.75 351.1492122573612 L 51.791666666666664 205.42771047357456 L 51.83333333333333 290.2208300403511 L 51.875 327.7137674720065 L 51.916666666666664 241.273825067859 L 51.958333333333336 307.36570619123654 L 52.0 307.36570619123654 L 52.041666666666664 265.9235831521652 L 52.08333333333333 387.3354220663217 L 52.125 317.57019818324466 L 52.166666666666664 324.33930199473684 L 52.208333333333336 347.8210176391559 L 52.25000000000001 357.78610667046036 L 52.291666666666664 354.47090013707145 L 52.333333333333336 255.4031606270123 L 52.37499999999999 320.95812115885536 L 52.416666666666664 317.57019818324466 L 52.45833333333333 205.42771047357456 L 52.5 251.88182711928408 L 52.54166666666667 303.95054361564286 L 52.583333333333336 297.0995552632812 L 52.62500000000001 347.8210176391559 L 52.666666666666664 276.3792642057342 L 52.708333333333336 244.817189875185 L 52.75 334.4426586801737 L 52.791666666666664 425.95169362769303 L 52.83333333333333 251.88182711928408 L 52.875 341.1450058698223 L 52.916666666666664 219.85703549429098 L 52.95833333333333 324.33930199473684 L 53.0 303.95054361564286 L 53.041666666666664 303.95054361564286 L 53.083333333333336 237.72305510403544 L 53.125 361.0948571013547 L 53.16666666666667 317.57019818324466 L 53.208333333333336 234.1648489673188 L 53.25000000000001 258.91721112332203 L 53.29166666666667 223.4453004168217 L 53.33333333333333 303.95054361564286 L 53.375 354.47090013707145 L 53.416666666666664 334.4426586801737 L 53.458333333333336 337.7971371820668 L 53.5 297.0995552632812 L 53.54166666666667 324.33930199473684 L 53.583333333333336 344.48629074076825 L 53.625 347.8210176391559 L 53.666666666666664 276.3792642057342 L 53.708333333333336 327.7137674720065 L 53.75 314.1755061262603 L 53.791666666666664 198.1668327167817 L 53.833333333333336 361.0948571013547 L 53.875 297.0995552632812 L 53.916666666666664 223.4453004168217 L 53.95833333333333 300.52850256093046 L 54.0 419.57629099030146 L 54.04166666666

## Homology in alignments

In [15]:
# reminder about homology in molecular world

<html>
<head>
</head>
<body>
	<table>
		<thead>
			<tr>
				<th>Ind.</th>
				<th>N1</th>
				<th>N2</th>
				<th>N3</th>
				<th>N4</th>
				<th>N5</th>
				<th>N6</th>
				<th>N7</th>
			</tr>
		</thead>
		<tbody>
			<tr>
				<td>Ind. 1</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#FC8D62;">C</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
			</tr>
			<tr>
				<td>Ind. 2</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
			</tr>
			<tr>
				<td>Ind. 3</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
			</tr>
			<tr>
				<td>Ind. 4</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
			</tr>          
		</tbody>
	</table>
</body>
</html>

calculate distance between two usually is highly underestimated.

Especially due to Superimposed substitution, 


Superimposed substitutions ("multiple hits"): The occurrence of two or more substitutions at the same site

<html>
<head>
</head>
<body>
	<table>
		<thead>
			<tr>
				<th>Time</th>
				<th>N1</th>
				<th>N2</th>
				<th>N3</th>
				<th>N4</th>
				<th>N5</th>
				<th>N6</th>
				<th>N7</th>
				<th>N8</th>
			</tr>
		</thead>
		<tbody>
			<tr>
				<td>1</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#FC8D62;">C</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#E78AC3;">T</td>
			</tr>
			<tr>
				<td>2</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#FC8D62;">C</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#E78AC3;">T</td>
			</tr>
			<tr>
				<td>3</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#E78AC3;">T</td>
			</tr>
			<tr>
				<td>4</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#E78AC3;">T</td>
			</tr>
			<tr>
				<td>5</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
			</tr>
		</tbody>
	</table>
    
    
</body>
</html>

Critical to models of nucleotide evolution is the realization that because there are only four possible character states, it is expected that as genetic distance increases, some sites will undergo multiple superimposed substitutions.

In this case, some sites that have undergone change will have reverted to the state they were originally in.
To reliably reconstruct evolutionary relationships among divergent sequences, this expected reversion must be taken into account.
Simple measures of distance that do not take multiple substitutions into account are said to be uncorrected. Corrected distances use one of several models of sequence evolution to estimate the number of sites that have undergone multiple substitutions.
Uncorrected methods tend to underestimate genetic distance.



<html>
<head>
</head>
<body>
	<table>
		<thead>
			<tr>
				<th>Species</th>
				<th>N1</th>
				<th>N2</th>
				<th>N3</th>
				<th>N4</th>
				<th>N5</th>
				<th>N6</th>
				<th>N7</th>
				<th>N8</th>
			</tr>
		</thead>
		<tbody>
			<tr>
				<td>Sp. 1</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#FC8D62;">C</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#E78AC3;">T</td>
			</tr>
			<tr>
				<td>Sp. 2</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#E78AC3;">T</td>
				<td style="background-color:#66C2A5;">A</td>
				<td style="background-color:#8DA0CB;">G</td>
				<td style="background-color:#66C2A5;">A</td>
			</tr>
		</tbody>
	</table>
    
    
</body>
</html>

Model asre important because they allow to correct to some extent the bias of underestimate differences when we are calculating distances between sequences froom observed data.44

show plot of expected vs observed. Here is where substitution models are important.



## Substitution models

Assumming that the sequences evolved under certain model. For example if all sites changed under the same probability or if some sites are more likely to change than others.  Based on these assumptions we can compare observed pattern against predicted expectations in each model and correct the distance.

They are Markov model 



 They have two parts:
 - Stationary distribution
 - Transition matrix
 

Based on these we can derive a formula to correct the distance bias, usally based in some known distribution, for example the  Poisson distribution.  
This distribution is conveneinte in some cases because it fits well where events are independents, continuous in time, and rare.

### Jukes-Cantor model

!!!!!!!Look for differences betwween transition matrix, rate matrix and instant transition matrix

Basic model, proposed by Jukes and Cantor (1969)
It has 2 assumbtions
- This assumes that there is only one substitution rate and it is equal for all bases
- All frequencies for all bases are equal (in the stationary distribution) 

This model is also know as a one paremeter model, because the model only needs one parameter.

In [219]:
# Define the only parameter
a = 0.333  # α (rate of substitution) 

In [220]:
# Transition matrix for Jukes-Cantor model
                 #  A        G     C     T
P = np.array([[1-3*a, a, a, a],  # A
                  [a, 1-3*a, a, a],  # G
                  [a, a, 1-3*a, a],  # C
                  [a, a, a, 1-3*a]   # T
                 ])

In [221]:
P

array([[0.001, 0.333, 0.333, 0.333],
       [0.333, 0.001, 0.333, 0.333],
       [0.333, 0.333, 0.001, 0.333],
       [0.333, 0.333, 0.333, 0.001]])

In [222]:
# Transition rate matrix for Jukes-Cantor model
                 #  A        G     C     T
Q = np.array([[-3*a, a, a, a],  # A
                  [a, -3*a, a, a],  # G
                  [a, a, -3*a, a],  # C
                  [a, a, a, -3*a]   # T
                 ])

In [223]:
Q

array([[-0.999,  0.333,  0.333,  0.333],
       [ 0.333, -0.999,  0.333,  0.333],
       [ 0.333,  0.333, -0.999,  0.333],
       [ 0.333,  0.333,  0.333, -0.999]])

In [19]:
get_stationary_dist_matrix_multiplication("ACGT", iterations=100, transition_matrix=jc_tm)

array([0.25, 0.25, 0.25, 0.25])

In [20]:
plot_convergency(states="ACGT", initial_state="C", steps= 1000, transition_matrix=jc_tm);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="teb64bd3e11ab456784c7d955b31ceed6"> <path d="M 50.0 550.0 L 50.48606358489392 550.0 L 50.972127169787846 550.0 L 51.45819075468176 550.0 L 51.944254339575686 550.0 L 52.430317924469605 550.0 L 52.91638150936353 550.0 L 53.40244509425745 550.0 L 53.88850867915137 550.0 L 54.37457226404529 550.0 L 54.86063584893922 550.0 L 55.34669943383314 466.6666666666667 L 55.832763018727064 473.07692307692304 L 56.31882660362098 407.14285714285717 L 56.804890188514904 416.6666666666667 L 57.29095377340882 362.5 L 57.77701735830275 373.5294117647058 L 58.26308094319667 383.33333333333337 L 58.74914452809059 392.10526315789474 L 59.23520811298451 350.0 L 59.72127169787842 359.5238095238095 L 60.20733528277235 368.1818181818182 L 60.693398867666275 332.608695652174 L 61.17946245256019 341.66666666666663 L 61.665526037454114 350.0 L 62.15158962234804 357.6923076923077 L 62.637653207241954 364.81481481481484 L 63.12371679213588 371.4285714285714 L 63.60978037702981 343.1034482758621 L 64.09584396192372 350.0 L 64.58190754681765 356.45161290322585 L 65.06797113171157 362.5 L 65.55403471660549 368.1818181818182 L 66.04009830149941 344.11764705882354 L 66.52616188639333 350.0 L 67.01222547128725 355.5555555555556 L 67.49828905618118 360.8108108108108 L 67.98435264107509 365.7894736842105 L 68.47041622596902 370.5128205128205 L 68.95647981086293 350.0 L 69.44254339575686 354.8780487804878 L 69.92860698065078 359.5238095238095 L 70.4146705655447 363.953488372093 L 70.90073415043862 368.1818181818182 L 71.38679773533255 372.2222222222222 L 71.87286132022646 376.0869565217391 L 72.35892490512039 379.78723404255317 L 72.84498849001432 383.33333333333337 L 73.33105207490823 386.734693877551 L 73.81711565980216 389.99999999999994 L 74.30317924469608 393.1372549019608 L 74.78924282959 396.15384615384613 L 75.27530641448391 399.0566037735849 L 75.76136999937785 401.8518518518519 L 76.24743358427176 404.54545454545456 L 76.73349716916567 407.14285714285717 L 77.2195607540596 392.10526315789474 L 77.70562433895353 394.82758620689657 L 78.19168792384744 397.45762711864404 L 78.67775150874137 400.0 L 79.1638150936353 386.0655737704919 L 79.6498786785292 388.7096774193549 L 80.13594226342313 391.26984126984127 L 80.62200584831706 393.75 L 81.10806943321097 396.15384615384613 L 81.5941330181049 383.33333333333337 L 82.08019660299883 385.820895522388 L 82.56626018789274 388.2352941176471 L 83.05232377278666 376.0869565217391 L 83.53838735768059 378.5714285714286 L 84.0244509425745 366.90140845070425 L 84.51051452746842 369.4444444444444 L 84.99657811236236 371.9178082191781 L 85.48264169725627 360.8108108108108 L 85.96870528215018 363.33333333333337 L 86.4547688670441 365.7894736842105 L 86.94083245193804 368.1818181818182 L 87.42689603683195 370.5128205128205 L 87.91295962172589 360.126582278481 L 88.3990232066198 362.5 L 88.88508679151371 364.81481481481484 L 89.37115037640764 354.8780487804878 L 89.85721396130157 357.2289156626506 L 90.34327754619548 359.5238095238095 L 90.8293411310894 361.7647058823529 L 91.31540471598333 352.3255813953488 L 91.80146830087725 354.59770114942535 L 92.28753188577116 345.4545454545454 L 92.7735954706651 347.75280898876406 L 93.25965905555901 350.0 L 93.74572264045293 341.2087912087912 L 94.23178622534685 343.4782608695652 L 94.71784981024078 345.6989247311828 L 95.20391339513469 337.2340425531915 L 95.68997698002863 339.4736842105263 L 96.17604056492254 341.66666666666663 L 96.66210414981646 343.8144329896907 L 97.14816773471038 335.71

The formula to correct the distance is:
$$D = -\frac{3}{4}ln(1 - \frac{4}{3}d)$$

Where ***D*** is the corrected distance and ***d*** is the distance between two sequences

In [239]:
def get_distances_jc(sequences, pair=(0,1), check_patterns=False):
    # tranform list of strings into arrays (useful to conect with random walk function)
    # and convert sequences in ascii values for easy diff calculation
    sequences_array = np.asarray([[ord(base) for base in sequence] for sequence in sequences])
    
    # Check patterns instead of individual sites
    if check_patterns:
        # simplify the alignment and only count the unique patterns (instead of sites) 
        # to calculate the likelihood of a given substitution
        patterns, pattern_frequencies = np.unique(sequences_array, axis=1, return_counts=True)

        # get differences
        differences = patterns[pair[0]] - patterns[pair[1]]
        
    # Get differences of nucleotides instead of patterns
    else:
        # check different bases
        differences = sequences_array[pair[0]] - sequences_array[pair[1]]

    # calculate the distance
    distance = np.count_nonzero(differences) / sequences_array.shape[1]

    # if distances is greater than 3/4 the equation does not exist
    if distance < 3/4:
        # correct distance using jc model
        corrected_distance = -3/4 * np.log(1 - 4/3 * distance)
    else:
        corrected_distance = np.inf
    
    return corrected_distance, distance

In [240]:
# Simple definition of sequences
sequences = ["GCTTCTGATTAACCTGCT",
             "GCTTCTGATTTCTCTGCC",
]

In [241]:
get_distances_jc(sequences, (0, 1))

(0.2635484151284164, 0.2222222222222222)

Let's generate a sequence under Jukes-Cantor model

In [242]:
sequence = print_random_walk(states="ACGT", 
                  steps=100, 
                  initial_state="C", 
                  transition_matrix=jc_tm)
sequence

'CTATCATGTCGTACGCACTCGACACTAGATGTAGCTGTGTATACGACAGACTGCAGAGTGCTGCACGAGTCACTAGAGATCTCATGTATACATGATATGA'

Now introduce 1000 random mutation and store each step to compare how different is the distance corrected from the corrected with the model

In [246]:
import random

generations = 1000
results = np.zeros((generations, 2))
np.random.seed(8)

sequence_as_array = np.asarray([base for base in sequence])

seq_evo_hist = [sequence_as_array]

for time in range(generations):
    
    new_sequence = np.copy(seq_evo_hist[time])
    
    if np.random.poisson() > 1:
        subtituted_site = np.random.randint(low=0, high=sequence_as_array.shape[0])
        subtitution = np.random.choice(list("ACGT"))
        new_sequence[subtituted_site] = subtitution

    seq_evo_hist.append(new_sequence)
    
#     corrected_distance, distance = get_distances_jc(["".join(seq_evo_hist[0]), "".join(seq_evo_hist[time])])
    corrected_distance, distance = get_distances_jc(["".join(i) for i in seq_evo_hist], (0,time))

    
    results[time] = [corrected_distance, distance]

canvas, axes, mark = toyplot.plot(results, ylabel="Distance", xlabel="Time");
axes.text(time, results[-1][0], "With model", style={"text-anchor":"start", "-toyplot-anchor-shift":"5px"});
axes.text(time, results[-1][1], "Without model", style={"text-anchor":"start", "-toyplot-anchor-shift":"5px"});

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1f12313e1bfc4891b3dafe5c3630534c"> <path d="M 50.0 550.0 L 50.41666666666667 548.4675727672052 L 50.83333333333333 548.4675727672052 L 51.25 548.4675727672052 L 51.66666666666667 548.4675727672052 L 52.08333333333333 546.9142955453653 L 52.5 546.9142955453653 L 52.916666666666664 546.9142955453653 L 53.33333333333333 546.9142955453653 L 53.75 546.9142955453653 L 54.16666666666667 546.9142955453653 L 54.58333333333333 546.9142955453653 L 55.0 546.9142955453653 L 55.416666666666664 546.9142955453653 L 55.83333333333333 546.9142955453653 L 56.25 546.9142955453653 L 56.66666666666667 546.9142955453653 L 57.08333333333333 546.9142955453653 L 57.5 546.9142955453653 L 57.916666666666664 546.9142955453653 L 58.33333333333333 546.9142955453653 L 58.75 546.9142955453653 L 59.16666666666667 546.9142955453653 L 59.58333333333333 546.9142955453653 L 60.0 545.33959313538 L 60.41666666666666 545.33959313538 L 60.83333333333333 545.33959313538 L 61.25 545.33959313538 L 61.66666666666667 545.33959313538 L 62.08333333333333 543.7428662023791 L 62.5 543.7428662023791 L 62.916666666666664 543.7428662023791 L 63.33333333333334 543.7428662023791 L 63.75 543.7428662023791 L 64.16666666666667 543.7428662023791 L 64.58333333333333 543.7428662023791 L 65.0 543.7428662023791 L 65.41666666666666 543.7428662023791 L 65.83333333333334 543.7428662023791 L 66.25 542.1234899062052 L 66.66666666666667 542.1234899062052 L 67.08333333333333 540.4808124333614 L 67.5 540.4808124333614 L 67.91666666666666 540.4808124333614 L 68.33333333333334 538.8141534217888 L 68.75 538.8141534217888 L 69.16666666666667 537.1228022689503 L 69.58333333333333 537.1228022689503 L 70.0 535.4060163126874 L 70.41666666666667 535.4060163126874 L 70.83333333333334 535.4060163126874 L 71.25 535.4060163126874 L 71.66666666666667 535.4060163126874 L 72.08333333333333 535.4060163126874 L 72.5 535.4060163126874 L 72.91666666666667 535.4060163126874 L 73.33333333333334 535.4060163126874 L 73.75 533.6630188731963 L 74.16666666666667 531.892997143202 L 74.58333333333333 531.892997143202 L 75.0 531.892997143202 L 75.41666666666667 531.892997143202 L 75.83333333333334 531.892997143202 L 76.25 531.892997143202 L 76.66666666666667 531.892997143202 L 77.08333333333333 531.892997143202 L 77.5 531.892997143202 L 77.91666666666667 531.892997143202 L 78.33333333333334 531.892997143202 L 78.75 530.0950999119832 L 79.16666666666667 530.0950999119832 L 79.58333333333333 530.0950999119832 L 80.0 530.0950999119832 L 80.41666666666667 530.0950999119832 L 80.83333333333333 528.2684351072976 L 81.25 528.2684351072976 L 81.66666666666667 528.2684351072976 L 82.08333333333333 528.2684351072976 L 82.5 526.4120671374392 L 82.91666666666666 526.4120671374392 L 83.33333333333333 526.4120671374392 L 83.75 524.5250140136 L 84.16666666666666 524.5250140136 L 84.58333333333333 522.6062442303822 L 85.0 522.6062442303822 L 85.41666666666667 522.6062442303822 L 85.83333333333334 522.6062442303822 L 86.25 522.6062442303822 L 86.66666666666666 522.6062442303822 L 87.08333333333334 522.6062442303822 L 87.5 522.6062442303822 L 87.91666666666667 522.6062442303822 L 88.33333333333333 522.6062442303822 L 88.75 522.6062442303822 L 89.16666666666666 522.6062442303822 L 89.58333333333334 522.6062442303822 L 90.0 522.6062442303822 L 90.41666666666666 522.6062442303822 L 90.83333333333333 522.6062442303822 L 91.25 522.6062442303822 L 91.66666666666666 522.6062442303822 L 92.08333333333333 522.6062442303822 L 92.5 522.6062442303822 L 92.916666

### Kimura 2 parameters models

this is a tests

### Gamma

### Invariant sites

# Archive

In [159]:
def get_probability_sequence(sequence, transition_matrix, stationary_distribution):
    initial_state = sequence[0]
    probability = pi[initial_state]
    previous_state = initial_state

    for i in range(1, len(sequence)):
        current_state = sequence[i]
        probability *= transition_matrix[previous_state][current_state]
        previous_state = current_state
        
    return probability

In [159]:
def get_probability_sequence(sequence, transition_matrix, stationary_distribution):
    initial_idx = 0
    probability = stationary_distribution[initial_idx]
    previous_state = sequence[initial_idx]

    for i in range(1, len(sequence)):
        current_state = sequence[i]
        probability *= transition_matrix[previous_state][current_state]
        previous_state = current_state
        
    return probability

In [ ]:
get_probability_sequence("ACCCCGGCCCCAAAGGGGGTATATAA", transition_matrix=transition_matrix, stationary_distribution=[0.25,0.25,0.25,0.25])

In [14]:
## comparing our distribution with a totally random sequence

In [ ]:
# The frequency of bases will be very different if the sequence that we have does fit into our markov model. For ecample

In [5]:
absolutly_random_sequence = "".join(np.random.choice(list("AGCT"), size=1000))
absolutly_random_sequence

'TTCAGTTCGATCGACAATACGTGGCTGTTACGCTTACGCGACGACAGTCTTACACAGGCACAAGTGCGTACGATCTGAGGCGTGGCATACGATCCTCGCGTGGGAGTGTACCGCGACTGATCGCCGCCAATCCTTTTAAGGCAGCGTGAGCTACCATATGACGGCGAAGGGGTAACATTCGTGTCGGACCGGTAATTTTGGATGGAGGACGTTGTCCGGTTTGGATAGTGGGACTTCGGGATGTTTACCACCGAACGTACTGACGAGGTCAGGGGAGTCGACTCAATATCAGAGGCGTACCTGATAAGGTTCCTACTACTCCTCGCGATTCCGGCGCAAAAGGATGTTCTAGGATAATGAATATTCTACCGGCAGCCGTGGACCGGTTGTCCGCATTTAGTAGATAGGGTGAATACATAACATAAGCCTCGTGGGGCCGCCCGGCGCTGAGTCACAGGCAAGGTAACTTAGTAGAGGGCGCGTGTGAACCCTGATCGATCGCGTGTTAGATCCGCCGGCGAGCGATATTCGGGCTAGATCATATGGACGCATTTGTTAACCGTTCGAGTCAGGGCCAAGAAGGTGGATCACATGTCGAGTTGTTTCATTTCGCCGACCCGTCTCAGCCGCAATGGCGAAAGCAAGGTGGTCTCGTGCGAGGCAGCGCCAAAAATCGGGTGGGACGACCCCTGGAACAATGTCGGGATGACATAAAACCAATTTAATGCCATACACGACGGCGCGAAGAGTTTGTGCAGCCCGCAATCGTATAAAAGGCCAGAAGCGTACGGCGTGATCCTGTTTGGTCACTATGCCCCATGCTTAGGGCGCACGCTTTTCCTCACCCGCTGGGAATACGGTATAAGCGCGCGAGCCCCCGCCGCGTTCTTAGGTCCCTAACTTAAGCTATTGCCCCCTGGATGAAGGGCGGTTCGTCACTTCTGTCCGGTCCCCTGGGCGCTTCATCTCCTACAGCATTTGGCTTCATGCGGGATCCCG

In [167]:
empiric_frequencies = {}
for base in "AGCT":
    empiric_frequencies[base] = absolutly_random_sequence.count(base) / len(absolutly_random_sequence)
    
empiric_frequencies  

{'A': 0.223, 'G': 0.291, 'C': 0.257, 'T': 0.229}

In [374]:

from __future__ import division
import math
import numpy as np

def JC69_Q():
    a = 0.01/3
    Q = np.asarray(
    [  -3*a, a, a, a,
       a, -3*a, a, a,
       a, a, -3*a, a,
       a, a, a, -3*a ])
    Q.shape = (4,4)
    return Q
    
def TN93_values():
    pi = [0.2,0.3,0.2,0.3]
    T,C,A,G = pi
    Y = T + C
    R = A + G
    a1 = 2   # Y transitions
    a2 = 1   # R transitions
    b = 0.2    # transversions
    return T,C,A,G,Y,R,a1,a2,b
    
def TN93_Q():
    T,C,A,G,Y,R,a1,a2,b = TN93_values()
    Q = np.asarray(
        [ -(a1*C + b*R), a1*C, b*A, b*G,
          a1*T, -(a1*T + b*R), b*A, b*G,
          b*T, b*C, -(a2*G + b*Y), a2*G,
          b*T, b*C, a2*A, -(a2*A + b*Y) ])
    Q.shape = (4,4)
    return Q
    
def TN93_P(t):
    # order TCAG
    T,C,A,G,Y,R,a1,a2,b = TN93_values()
    e2 = math.e**(-b*t)
    e3 = math.e**(-(R*a2 + Y*b)*t)
    e4 = math.e**(-(Y*a1 + R*b)*t)

    TY,CY,AR,GR = T/Y,C/Y,A/R,G/R
    M = [
        T + TY*R*e2 + CY*e4, C + CY*R*e2 - CY*e4,
        A*(1 - e2), G*(1 - e2),
        T + TY*R*e2 - TY*e4, C + CY*R*e2 + TY*e4,
        A*(1 - e2), G*(1 - e2),
        T*(1 - e2), C*(1 - e2),
        A + AR*Y*e2 + GR*e3, G + GR*Y*e2 - GR*e3,
        T*(1 - e2), C*(1 - e2),
        A + AR*Y*e2 - AR*e3, G + GR*Y*e2 + AR*e3  ]
    P = np.array(M)
    P.shape = (4,4)
    return P
        
def convert(Q,t,debug=False):
    if debug:
        print ('Q', '\n', Q)
    evals, evecs = np.linalg.eig(Q)
    if debug:
        print  ('evals', '\n', evals)
    L = np.diag([math.e**(k*t) for k in evals])
    if debug:
        print  ('L', '\n', L)
    U = evecs
    if debug:
        print  ('U', '\n', U)
    iU = np.linalg.inv(U)
    if debug:
        print  ('iU', '\n', iU)
    P = np.dot(U,np.dot(L,iU))
    if debug:
        print  ('P', '\n', P)
    return P
    
def long_time(P,N=5000):
    M = P
    for i in range(N):
        M = np.dot(P,M)
    return M

In [ ]:
convert(JC69_Q(), 10000, debug=True)

Q 
 [[-0.01        0.00333333  0.00333333  0.00333333]
 [ 0.00333333 -0.01        0.00333333  0.00333333]
 [ 0.00333333  0.00333333 -0.01        0.00333333]
 [ 0.00333333  0.00333333  0.00333333 -0.01      ]]
evals 
 [-1.33333333e-02  4.33680869e-19 -1.33333333e-02 -1.33333333e-02]
L 
 [[1.24184982e-58 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.24184982e-58 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.24184982e-58]]
U 
 [[-0.8660254   0.5        -0.2747774  -0.12962158]
 [ 0.28867513  0.5         0.7841548   0.40424231]
 [ 0.28867513  0.5         0.04519959 -0.76264153]
 [ 0.28867513  0.5        -0.55457698  0.4880208 ]]
iU 
 [[-0.8660254  -0.01093835  0.42842131  0.44854245]
 [ 0.5         0.5         0.5         0.5       ]
 [-0.          0.77008713 -0.05158606 -0.71850107]
 [-0.          0.36930849 -0.82431531  0.45500682]]
P 
 [[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])